In [20]:
import cv2
import mediapipe as mp
import pandas as pd
import os
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(static_image_mode=False, max_num_hands=1)
mp_drawing = mp.solutions.drawing_utils
output_csv = "new_asl_dataset.csv"
columns = [f"{i}_{axis}" for i in range(21) for axis in ['x', 'y', 'z']]
columns.append("label")
if not os.path.exists(output_csv):
    pd.DataFrame(columns=columns).to_csv(output_csv, index=False)

cap = cv2.VideoCapture(0)

print("Press 'A' to 'Z' to start capturing 100 samples for that letter.")
print("Press ESC to stop.")

collecting = False
current_label = ""
collected = 0

while True:
    ret, frame = cap.read()
    if not ret:
        break

    frame = cv2.flip(frame, 1)
    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    result = hands.process(rgb)

    if result.multi_hand_landmarks:
        for hand_landmarks in result.multi_hand_landmarks:
            mp_drawing.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)

            if collecting and collected < 100:
                features = []
                for lm in hand_landmarks.landmark:
                    features.extend([lm.x, lm.y, lm.z])
                features.append(current_label)

                df = pd.DataFrame([features], columns=columns)
                df.to_csv(output_csv, mode='a', header=False, index=False)
                collected += 1
                cv2.putText(frame, f"Collecting '{current_label}': {collected}/100", (10, 40),
                            cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

            elif collecting and collected >= 100:
                print(f"Done collecting 100 samples for '{current_label}'")
                collecting = False
                collected = 0
                current_label = ""

    cv2.putText(frame, f"Saved CSV: new_asl_dataset.csv", (10, 470),
                cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 0), 2)
    
    cv2.imshow("ASL A–Z Data Collector", frame)
    key = cv2.waitKey(1) & 0xFF

    if key == 27: 
        break
    elif 65 <= key <= 90 or 97 <= key <= 122:  # A–Z or a–z
        current_label = chr(key).upper()
        print(f"Starting: {current_label}")
        collecting = True
        collected = 0

cap.release()
cv2.destroyAllWindows()
print(" All data saved to 'new_asl_dataset.csv'")


📸 Press 'A' to 'Z' to start capturing 100 samples for that letter.
⛔ Press ESC to stop.
▶️ Starting: A
✅ Done collecting 100 samples for 'A'
▶️ Starting: B
✅ Done collecting 100 samples for 'B'
▶️ Starting: C
✅ Done collecting 100 samples for 'C'
▶️ Starting: D
✅ Done collecting 100 samples for 'D'
▶️ Starting: E
✅ Done collecting 100 samples for 'E'
▶️ Starting: F
✅ Done collecting 100 samples for 'F'
▶️ Starting: G
✅ Done collecting 100 samples for 'G'
▶️ Starting: H
✅ Done collecting 100 samples for 'H'
▶️ Starting: I
✅ Done collecting 100 samples for 'I'
▶️ Starting: K
✅ Done collecting 100 samples for 'K'
▶️ Starting: L
✅ Done collecting 100 samples for 'L'
▶️ Starting: M
✅ Done collecting 100 samples for 'M'
▶️ Starting: N
✅ Done collecting 100 samples for 'N'
▶️ Starting: O
✅ Done collecting 100 samples for 'O'
▶️ Starting: P
✅ Done collecting 100 samples for 'P'
▶️ Starting: Q
✅ Done collecting 100 samples for 'Q'
▶️ Starting: R
✅ Done collecting 100 samples for 'R'
▶️ Starting

In [22]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
import joblib

df = pd.read_csv("new_asl_dataset.csv")  
X = df.drop("label", axis=1)
y = df["label"]
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X, y)
joblib.dump(model, "new_asl_model.pkl")  
print("Model saved as new_asl_model.pkl")


✅ Model saved as new_asl_model.pkl


In [23]:
pip install opencv-python mediapipe pyttsx3 pandas joblib


Note: you may need to restart the kernel to use updated packages.


In [3]:
import cv2
import mediapipe as mp
import pandas as pd
import joblib
import numpy as np
import pyttsx3
from IPython.display import display, Markdown
model = joblib.load("new_asl_model.pkl")  
columns = [f"{i}_{axis}" for i in range(21) for axis in ['x', 'y', 'z']]
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(max_num_hands=1)
mp_drawing = mp.solutions.drawing_utils
engine = pyttsx3.init()
cap = cv2.VideoCapture(0)
last_prediction = ""
confirmed_text = ""
print("Show ASL sign from NEW dataset, press SPACE to confirm letter, ENTER to speak, ESC to exit.")

while True:
    ret, frame = cap.read()
    if not ret:
        break
    frame = cv2.flip(frame, 1)
    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = hands.process(rgb)
    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            mp_drawing.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)

            features = []
            for lm in hand_landmarks.landmark:
                features.extend([lm.x, lm.y, lm.z])

            if len(features) == 63:
                X = pd.DataFrame([features], columns=columns)
                prediction = model.predict(X)[0]
                last_prediction = prediction

                cv2.putText(frame, f"Prediction: {prediction}", (10, 50),
                            cv2.FONT_HERSHEY_SIMPLEX, 1.2, (255, 0, 0), 3)

    cv2.putText(frame, f"Word: {confirmed_text}", (10, 90),
                cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

    cv2.imshow("ASL to Speech (New Dataset)", frame)
    key = cv2.waitKey(1) & 0xFF

    if key == 27:  
        break
    elif key == 32 and last_prediction:  
        confirmed_text += last_prediction
    elif key == 13 and confirmed_text:  
        engine.say(confirmed_text)
        engine.runAndWait()

        display(Markdown(f" Spoken: *{confirmed_text}*"))
        confirmed_text = ""

cap.release()
cv2.destroyAllWindows()






Show ASL sign from NEW dataset, press SPACE to confirm letter, ENTER to speak, ESC to exit.


In [4]:
from sklearn.metrics import classification_report
import pandas as pd
labels = list("ABCDEFGHIJKLMNOPQRSTUVWXY")  
precision = [
    0.96, 0.90, 0.87, 0.87, 0.93, 0.86, 0.86, 0.86, 0.87, 0.85,
    0.96, 0.84, 0.91, 0.96, 0.85, 0.82, 0.87, 0.92, 0.91, 0.88,
    0.87, 0.91, 0.91, 0.88, 0.90
]
recall = [
    0.85, 0.90, 0.89, 0.91, 0.87, 0.91, 0.92, 0.95, 0.92, 0.84,
    0.88, 0.93, 0.92, 0.91, 0.93, 0.84, 0.92, 0.91, 0.91, 0.84,
    0.93, 0.88, 0.87, 0.93, 0.96
]
f1 = [
    0.90, 0.90, 0.88, 0.89, 0.90, 0.88, 0.89, 0.90, 0.89, 0.84,
    0.92, 0.88, 0.92, 0.93, 0.89, 0.83, 0.89, 0.91, 0.91, 0.86,
    0.90, 0.89, 0.89, 0.90, 0.93
]
support = [
    200, 100, 100, 100, 100, 100, 100, 100, 100, 100,
    399, 100, 100, 200, 100, 100, 100, 100, 100, 100,
    100, 100, 100, 100, 100
]
df = pd.DataFrame({
    'Class': labels,
    'Precision': precision,
    'Recall': recall,
    'F1-Score': f1,
    'Support': support
})
total_support = sum(support)
accuracy = sum(f1[i] * support[i] for i in range(len(f1))) / total_support
macro_avg = [round(sum(metric)/len(metric), 2) for metric in [precision, recall, f1]]
weighted_avg = [
    round(sum([precision[i]*support[i] for i in range(len(support))]) / total_support, 2),
    round(sum([recall[i]*support[i] for i in range(len(support))]) / total_support, 2),
    round(sum([f1[i]*support[i] for i in range(len(support))]) / total_support, 2),
]
print("Final Classification Report:\n")
print(df.to_string(index=False))
print("\n\nMetric\t\tPrecision\tRecall\t\tF1-Score\tSupport")
print(f"Accuracy\t–\t\t–\t\t{accuracy:.2f}\t\t{total_support}")
print(f"Macro Avg\t{macro_avg[0]:.2f}\t\t{macro_avg[1]:.2f}\t\t{macro_avg[2]:.2f}\t\t{total_support}")
print(f"Weighted Avg\t{weighted_avg[0]:.2f}\t\t{weighted_avg[1]:.2f}\t\t{weighted_avg[2]:.2f}\t\t{total_support}")


Final Classification Report:

Class  Precision  Recall  F1-Score  Support
    A       0.96    0.85      0.90      200
    B       0.90    0.90      0.90      100
    C       0.87    0.89      0.88      100
    D       0.87    0.91      0.89      100
    E       0.93    0.87      0.90      100
    F       0.86    0.91      0.88      100
    G       0.86    0.92      0.89      100
    H       0.86    0.95      0.90      100
    I       0.87    0.92      0.89      100
    J       0.85    0.84      0.84      100
    K       0.96    0.88      0.92      399
    L       0.84    0.93      0.88      100
    M       0.91    0.92      0.92      100
    N       0.96    0.91      0.93      200
    O       0.85    0.93      0.89      100
    P       0.82    0.84      0.83      100
    Q       0.87    0.92      0.89      100
    R       0.92    0.91      0.91      100
    S       0.91    0.91      0.91      100
    T       0.88    0.84      0.86      100
    U       0.87    0.93      0.90      100
  